In [1]:
# INFO 
# https://coderoad.ru/44853059/Tensorflow-%D1%81%D0%BE%D0%BE%D0%B1%D1%89%D0%B5%D0%BD%D0%B8%D1%8F-%D0%B6%D1%83%D1%80%D0%BD%D0%B0%D0%BB%D0%B0-%D0%BD%D0%B5-%D0%BE%D1%82%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B0%D1%8E%D1%82%D1%81%D1%8F
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
[print(x) for x in os.environ["PATH"].split(";")];

C:\Users\Jaroslav\Desktop\Job\Python\DS\CoronaHack\venv\Scripts
C:\Python\Python38\Scripts\
C:\Python\Python38\
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin
C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\libnvvp
C:\Python\Python39\Scripts\
C:\Python\Python39\
C:\WINDOWS\system32
C:\WINDOWS
C:\WINDOWS\System32\Wbem
C:\WINDOWS\System32\WindowsPowerShell\v1.0\
C:\WINDOWS\System32\OpenSSH\
C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common
C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR
C:\Go\bin
C:\Program Files\Git\cmd
C:\Program Files\Perforce\
C:\WINDOWS\system32
C:\WINDOWS
C:\WINDOWS\System32\Wbem
C:\WINDOWS\System32\WindowsPowerShell\v1.0\
C:\WINDOWS\System32\OpenSSH\
C:\Program Files\NVIDIA Corporation\Nsight Compute 2020.3.1\
C:\Users\Jaroslav\Python\Python38\Scripts\
C:\Users\Jaroslav\Python\Python38
C:\Users\Jaroslav\Desktop\Job\Bazel
C:\Users\Jaroslav\AppData\Local\Microsoft\WindowsApps


In [6]:
%%time
#print("Time import CV2 = ", end='')
#%time import cv2
print("Time import Numpy = ", end='')
%time import numpy as np
print("Time import Pandas = ", end='')
%time import pandas as pd
print("Time import matplotlib.pyplot = ", end='')
%time from matplotlib import pyplot as plt
print("Time import seaborn = ", end='')
%time import seaborn as sns
print("Time import tqdm = ", end='')
%time from tqdm import tqdm
#print("Time import sklearn.decomposition.PCA = ", end='')
#%time from sklearn.decomposition import PCA

#For normal work ipywidgets
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#print("Time import ipywidgets.interact and ipywidgets.interact_manual = ", end='')
#%time from ipywidgets import interact, interact_manual

print("Time import tensorflow = ", end='')
%time import tensorflow as tf
print("Time import tensorflow.keras = ", end='')
%time import tensorflow.keras as keras
print("Time import tensorflow.keras.layers = ", end='')
%time from tensorflow.keras import layers

# For work using XLA
tf.config.optimizer.set_jit(True)

# For graphs display in JupyterNotebook
%matplotlib inline

pd.options.mode.chained_assignment = None  # For disable SettingWithCopyWarning, arising from the ambiguity of change
path = %pwd
train_path = path + "\\Coronahack-Chest-XRay-Dataset\\Coronahack-Chest-XRay-Dataset\\train\\"
test_path = path + "\\Coronahack-Chest-XRay-Dataset\\Coronahack-Chest-XRay-Dataset\\test\\"

NUM_DISEASE_TYPE = 7

# If (fast == True) used precompile variable mean image size; else this variable is calculated now
fast = True

# TODO:
#  Add the ability to specify a percentage of the RAM of the device on which notepad is running
#  For example:
#   There are only 8 GB of RAM on the machine, we indicate the number 50 corresponding to 50%,
#   then 4 will be entered into the variable; and if 32 GB and 25% then the variable will be 8

num_GB = 5 # Number of gigabytes of RAM allocated for storing images data
img_type = np.float64 # The type in which the image data will be stored
tf.config.list_physical_devices()

Time import Numpy = Wall time: 312 ms
Time import Pandas = Wall time: 478 ms
Time import matplotlib.pyplot = Wall time: 520 ms
Time import seaborn = Wall time: 709 ms
Time import tqdm = Wall time: 11 ms
Time import tensorflow = Wall time: 3.77 s
Time import tensorflow.keras = Wall time: 0 ns
Time import tensorflow.keras.layers = Wall time: 0 ns
Wall time: 5.92 s


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Read and pretreatment data 

In [7]:
data = pd.read_csv(path + '\\Chest_xray_Corona_Metadata.csv', sep=',')

In [8]:
data.drop("Unnamed: 0", axis=1, inplace=True) # Delete "Unnamed: 0" column because it enumerating objects

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910 entries, 0 to 5909
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   X_ray_image_name        5910 non-null   object
 1   Label                   5910 non-null   object
 2   Dataset_type            5910 non-null   object
 3   Label_2_Virus_category  69 non-null     object
 4   Label_1_Virus_category  4334 non-null   object
dtypes: object(5)
memory usage: 231.0+ KB


In [10]:
print(set(data["Label"]))
print(set(data["Label_1_Virus_category"]))
print(set(data["Label_2_Virus_category"]))

{'Normal', 'Pnemonia'}
{nan, 'Stress-Smoking', 'bacteria', 'Virus'}
{nan, 'COVID-19', 'SARS', 'Streptococcus', 'ARDS'}


In [11]:
def gen(df: pd.DataFrame) -> pd.Series:
    res = pd.Series(range(df.shape[0]), dtype=np.uint8)
    for i in range(df.shape[0]):
        if df["Label"][i] == "Normal":
            res[i] = 0 # "Normal"
            continue
        if df["Label_1_Virus_category"][i] == "Stress-Smoking":
            res[i] = 1 # "Stress-Smoking"
            continue
        if df["Label_1_Virus_category"][i] == "Virus":
            if df["Label_2_Virus_category"][i] == "COVID-19":
                res[i] = 2 # "COVID-19"
                continue
            if df["Label_2_Virus_category"][i] == "SARS":
                res[i] = 3 # "SARS"
                continue
            res[i] = 4 # "Undefined Virus"
            continue
        if df["Label_1_Virus_category"][i] == "bacteria":
            if df["Label_2_Virus_category"][i] == "Streptococcus":
                res[i] = 5 # "Streptococcus"
                continue
            res[i] = 6 # "Undefined Bacteria"
            continue
    return res.astype("category")
data = data.assign(Disease_type=gen)
data = data.drop(["Label", "Label_1_Virus_category", "Label_2_Virus_category"], axis=1)

In [12]:
train_df = data[data["Dataset_type"] == "TRAIN"].drop(["Dataset_type"], axis=1)
test = data[data["Dataset_type"] == "TEST"].drop(["Dataset_type"], axis=1)

In [13]:
%%time
if not fast:
    shapes = set()
    train_df["X_ray_image_name"].map(
        lambda img: shapes.add(cv2.imread(train_path + img, cv2.IMREAD_GRAYSCALE).shape)
    )
    shapes = np.array(list(shapes))
    
    min_img_size  = tuple(map(lambda x: int(x.min()),  shapes.T))
    mean_img_size = tuple(map(lambda x: int(x.mean()), shapes.T))
    max_img_size  = tuple(map(lambda x: int(x.max()),  shapes.T))
else:
    min_img_size = (127, 255)
    mean_img_size = (1008, 1354)
    max_img_size = (3480, 4248)
print(f"Min image size = {min_img_size}")
print(f"Average image size = {mean_img_size}")
print(f"Max image size = {max_img_size}")
print("Old time if not fast = 1min 30s")
print("New time if not fast use np = 1min 12s")

Min image size = (127, 255)
Average image size = (1008, 1354)
Max image size = (3480, 4248)
Old time if not fast = 1min 30s
New time if not fast use np = 1min 12s
Wall time: 3 ms


In [14]:
%%time
if not fast:
    img_sizes = set()
    for img in tqdm(train_df["X_ray_image_name"]):
        img_d = cv2.imread(train_path + img, cv2.IMREAD_GRAYSCALE)
        img_sizes.add(img_d.shape)
        
    sum_img_sizes = [0, 0]
    for i, j in img_sizes:
        sum_img_sizes[0] += i
        sum_img_sizes[1] += j
    mean_img_size = tuple(int(i / len(img_sizes)) for i in sum_img_sizes)
else:
    mean_img_size = (1008, 1354) # Counted before
print(f"Average image size = {mean_img_size}")

Average image size = (1008, 1354)
Wall time: 0 ns


In [15]:
train = train_df.sample(frac=0.7, random_state=1).reset_index(drop=True)
valid = train_df.drop(train.index).reset_index(drop=True)

In [16]:
class ImgGen:
    _first = True
    
    def __new__(cls, df, *args, reset=False, epochs=10, batch_size=16, **kwargs):
        cls._data = df
        if reset:
            cls._first = reset
        if cls._first:
            print(1)
            data = iter(
                tf.data.Dataset.range(epochs)
                .interleave(
                    cls._dataset_gen,
                    num_parallel_calls=tf.data.AUTOTUNE,
                )
                .map(
                    cls._read,
                    num_parallel_calls=tf.data.AUTOTUNE
                )
                .map(
                    cls._resize,
                    num_parallel_calls=tf.data.AUTOTUNE
                )
                .batch(batch_size)
            )
            cls._first = False
            cls.__obj = data
            return data#[next(data) for _ in range(batch_size)]
        else:
            print(9)
            return cls.__obj
            
        
    @classmethod
    def _dataset_gen(cls, *args):
        return tf.data.Dataset.from_generator(
            cls._gen,
            output_signature=(
                tf.TensorSpec(
                    shape=(),
                    dtype=tf.string
                ),
                tf.TensorSpec(
                    shape=(7, ),
                    dtype=tf.float16
                )
            )
        )

    @classmethod
    def _gen(cls):
        for img_name, label in cls._data[:2].values:
            label_tf_categ = np.zeros(NUM_DISEASE_TYPE, dtype=np.float16) # Label in tf categorical type
            label_tf_categ[label] = 1.
            yield tf.constant(train_path + img_name), tf.constant(label_tf_categ)

    @classmethod
    def _read(cls, img_path: tf.string, label: tf.int8):
        image = tf.io.read_file(img_path)
        image = tf.image.decode_jpeg(image, channels=1, )
        return image, label

    @classmethod
    def _resize(cls, x, label: tf.int8):
        return tf.image.resize(x, max_img_size, method="area"), label

In [17]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(*max_img_size, 1)),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.Dropout(.3),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    #layers.Dense(64, activation="relu"),
    #layers.Dropout(.3),
    #layers.Dense(32, activation="relu"),
    #layers.Dropout(.3),
    #layers.Dense(64, activation="relu"),
    #layers.Dropout(.3),
    layers.Dense(7, activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3478, 4246, 32)    320       
_________________________________________________________________
dropout (Dropout)            (None, 3478, 4246, 32)    0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1739, 2123, 32)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1737, 2121, 32)    9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1737, 2121, 32)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1735, 2119, 32)    9248      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1735, 2119, 32)    0

In [18]:
%%time
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

Wall time: 18 ms


In [ ]:
train_gen = ImgGen(train, reset=True, batch_size=1, epochs=2)
#valid_gen=

model.fit(
    train_gen,
    #validation_data=valid_gen,
    epochs=2,
    #use_multiprocessing=True,
    verbose=1,
    workers=16,
)

1
Epoch 1/2


In [ ]:
raise Warning()

In [ ]:
class ImgGen(keras.utils.Sequence):
    def __init__(self, df, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.data = df["X_ray_image_name"].T
        self.labels = df["Disease_type"]
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]

        # Find Data Frame
        df_temp = [self.data[k] for k in indexes]
        
        # Generate data
        X, y = self.__data_generation(df_temp)

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, df_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        size = self.batch_size
        
        
        # Initialization
        X = np.empty((size, *self.dim, self.n_channels), dtype=np.uint8)
        y = np.empty(size, dtype=np.uint8)
        
        # Generate data
        for i, ID in enumerate(df_temp):
            # Store sample         
            img = cv2.imread(train_path + ID, cv2.IMREAD_GRAYSCALE)
            rsz_img = cv2.resize(img, (mean_img_size[1], mean_img_size[0]), interpolation=cv2.INTER_AREA)
            X[i,] = (rsz_img / 255).reshape((*rsz_img.shape, 1))

            # Store class
            y[i] = self.labels[i]
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
%%time
params_for_ImgGen = {
    'dim': mean_img_size,
    'batch_size': 8,
    'n_classes': 7,
    'n_channels': 1,
    'shuffle': True,
}

training_generator = ImgGen(train, **params_for_ImgGen)
validation_generator = ImgGen(valid, **params_for_ImgGen)

n = train.shape[0]
n0, n1, n2, n3, n4, n5, n6 = 0, 0, 0, 0, 0, 0, 0
for i in train["Disease_type"]:
    if i == 0:
        n0 += 1
    elif i == 1:
        n1 += 1
    elif i == 2:
        n2 += 1
    elif i == 3:
        n3 += 1
    elif i == 4:
        n4 += 1
    elif i == 5:
        n5 += 1
    else:
        n6 += 1
class_weight={
    0: 1-n0/n,
    1: 1-n1/n,
    2: 1-n2/n,
    3: 1-n3/n,
    4: 1-n4/n,
    5: 1-n5/n,
    6: 1-n6/n,
}

model.fit(
    training_generator,
    validation_data=validation_generator,
    class_weight=class_weight,
    epochs=10,
    verbose=1,
    use_multiprocessing=True,
    workers=16,
)

In [ ]:
model.save("my_model.h5")

In [ ]:
raise Warning()

In [ ]:
model = keras.models.load_model("my_model.h5")

In [ ]:
y = mean_img_size[0] * mean_img_size[1]
x = int(num_GB * (1024 ** 3) / (img_type().itemsize * y)) # (1024 ** 3) - number byte in gigabyte
img_data = np.zeros((x, y), dtype=img_type)

In [ ]:
for i, img_name in tqdm(enumerate(train["X_ray_image_name"][:x])):
    img = cv2.imread(train_path + img_name, cv2.IMREAD_GRAYSCALE)
    rsz_img = cv2.resize(img, mean_img_size, interpolation=cv2.INTER_AREA)
    img_d = rsz_img.ravel() / 255
    img_data[i] = img_d

In [ ]:
img_data.shape

In [ ]:
pd_img_d = pd.DataFrame(img_data)

In [ ]:
img_data_T = img_data.T

In [ ]:
@interact_manual
def _(x="", y=""):
    sns.distplot(pd_img_d[:int(x)][:int(y)])
    print(x)

In [ ]:
sns.pairplot(pd_img_d[:3][:3])

In [ ]:
%%time
pca = PCA(n_components=2)
pca_t = pca.fit_transform(pd_img_d.T)

In [ ]:
np.save("./pca_t1.npy", pca_t)

In [ ]:
pca_t = np.load("./pca_t.npy")

In [ ]:
%%time
pca_1 = PCA(n_components=1)
pca_t_1 = pca_1.fit_transform(pca_t)

In [ ]:
pca_t_1.shape

In [ ]:
pca_t_1 = pca_t_1.T * 255
plt.imshow(pca_t_1.reshape(mean_img_size))

In [ ]:
%%time
sns.heatmap(pd_img_d.corr())

In [ ]:
print("unic CoD = ", unic_CoD, "\n")
print("unic ToD = ", unic_ToD, "\n")
@interact
def test(x=data.columns[1:], y=data.columns[1:]):
    return pd.crosstab(data[x], data[y])

In [ ]:
test["Cause_of_disease"].unique() # => "Cause_of_disease" target feature

In [ ]:
model = keras

# All information on program

In [ ]:
(x, y)